# Fourth Project PSS

## Ghadamiyan Lida

This project consists in the implementation of the Min Max algorithm in the context of the TicTacToe game.

How does it work:

At each step the algorithm computes all the possible moves (depending on the given depth) until it reaches a final state or the maximum depth if you are playing an easy level. It computes the score for each move considering that both players will play their best game, taking the moves that maximizez his score and minimizes the score of the opponent. When reaching the leaves we go upwards in the tree of moves and assigning the max score of the subtrees if that's the algorithm's turn or the minimum otherwise.

In [1]:
def is_winner(line):                                     #   check if a line decides the winner of the game
    
    countx = 0
    count0 = 0
    
    for element in line:
        
        if element == 'x':         # get the nr of x's and 0's
            countx += 1
            
        elif element == '0':
            count0 += 1
    
    if countx != 3 and count0 != 3:      # return the winner if there is one
        return False
    else:
        return line[0]

In [2]:
class Game:
    
    pmin = 'x'     ####### the human
    pmax = '0'    ####### algorithm
    mty = '_'
    
    def __init__(self, table = None):
        self.matrix = table or [self.__class__.mty]*9   
        
    def check(self):                                       # check if the game is ended - if there is any winner line
        winner = (is_winner(self.matrix[0:3])       #   
               or is_winner(self.matrix[3:6])       # horizontal config
               or is_winner(self.matrix[6:9])       #
               or is_winner(self.matrix[0:9:3])  #
               or is_winner(self.matrix[1:9:3])  # vertical config
               or is_winner(self.matrix[2:9:3])  #
               or is_winner(self.matrix[0:9:4])     #  diagonals
               or is_winner(self.matrix[2:8:2]))    # 
        
        if(winner):
            return str(winner)
        elif '_' not in self.matrix:              # return the winner or TOE
            return "TOE"
        else:
            return False
        
    def moves(self, op):             # possible configs after a move            
        moves = []
        
        for i in range(9):
            if self.matrix[i] == self.__class__.mty:   # checks for any empty spots and marks them as possible moves
                new_config = list(self.matrix)
                new_config[i] = op
                moves.append(Game(new_config))
        return moves
    
    def open_line(self, line, player):    # check if a line contains mt spots for move
        if len(set(line)) <= 2:
            if (self.__class__.mty in set(line)):
                return 1
            else:
                return 0
        else:
            return 0
                
    
    def open_lines(self, player):      # check if there is an open line
        return (self.open_line(self.matrix[0:3], player)
               +self.open_line(self.matrix[3:6], player)
               +self.open_line(self.matrix[6:9], player)
               +self.open_line(self.matrix[0:9:3], player)
               +self.open_line(self.matrix[1:9:3], player)
               +self.open_line(self.matrix[2:9:3], player)
               +self.open_line(self.matrix[0:9:4], player)
               +self.open_line(self.matrix[2:8:2], player))
    
    def get_score(self, depth):     # get an approx score   taking intro account the depth => sooner win > later win 
        fin = self.check()
        
        if fin == self.__class__.pmax:  # maximize when the algorithm wins   
            return(99+depth)
        
        elif fin == self.__class__.pmin:  # minimize when the human wins
            return(-99-depth)
        
        else:
            return 0
        
        
    def __str__(self):
        return (str([x for x in self.matrix[0:3]])+'\n'           # get the game table
               +str([x for x in self.matrix[3:6]])+'\n'
               +str([x for x in self.matrix[6:9]])+'\n')
        

In [3]:
class State:
    
    def __init__(self, table, player, depth, parent=None, score=None):       
        self.table = table
        self.player = player
        self.depth = depth           # details about the game
        self.score = score
        self.poss_moves = []
        self.best_state = None
        
    def opposite_player(self):          # return the opposite player
        if self.player == Game.pmin:
            return Game.pmax
        else:
            return Game.pmin
        
    def moves(self):   
        moves = self.table.moves(self.player)   # possible table configs
        opposite_player = self.opposite_player()
        moves = [State(move, opposite_player, self.depth-1, parent = self) for move in moves]
        return moves
    
    def __str__(self):
        return str(self.table)
    

In [4]:
def min_max(state):
    
    if state.depth == 0 or state.table.check():           # final state
        state.score = state.table.get_score(state.depth)
        return state
    
    state.poss_moves = state.moves()    # min max on every possible move
    moves_score = [min_max(move) for move in state.poss_moves]     
    
    if state.player == Game.pmax:       # if it's the algorithm's turn we choose the net state which has the max score
        state.best_state = max(moves_score, key=lambda x: x.score)
        
    else:     # else we choose the next state which has the min score / assuming this is what the opponen will choose
        state.best_state = min(moves_score, key=lambda x: x.score)
    
    state.score = state.best_state.score
    return state

In [5]:
def show_(current_state):
    
    ck = current_state.table.check()
    if ck:
        print(ck)
        return True
    return False

In [6]:
def main():
    
    current_table = Game();
    print('Current Table')
    print(str(current_table))
    
    print("Choose the dificulty level:\n1.Easy\n2.Medium\n3.Hard")
    level = int(input('level:'))
    
    if level == 1:
        current_state = State(current_table, 'x', 1)
    elif level == 2:
        current_state = State(current_table, 'x', 3)
    else:
        current_state = State(current_table, 'x', 30)
    
    while True:
        
        if current_state.player == Game.pmin:
            valid = False
            while not valid:
                try:
                    line = int(input('row: '))
                    col = int(input('col: '))
                    
                    if (line in range(0, 3)) and (col in range(0, 3)):
                        if current_state.table.matrix[line*3+col] == Game.mty:
                            valid = True
                        
                        else:
                            print("Chose an empty spot")
                    else:
                        print("Choose valid input: 0 1 2")
                        
                except ValueError:
                    print("Choose a valid input: 0 1 2")
                    
            current_state.table.matrix[line*3+col] = Game.pmin
            
            print('\n Game table:')
            print(str(current_state))
            
            if show_(current_state):
                break
            
            current_state.player = current_state.opposite_player()
            
        else:
            
            a_state = min_max(current_state)
            a_state.table = a_state.best_state.table
            print('\n Game table:')
            print(str(current_state))
            
            if show_(current_state):
                break
            
            current_state.player = current_state.opposite_player()
                            
                        

In [7]:
if __name__ == '__main__':
    main()

Current Table
['_', '_', '_']
['_', '_', '_']
['_', '_', '_']

Choose the dificulty level:
1.Easy
2.Medium
3.Hard
level:1
row: 0
col: 0

 Game table:
['x', '_', '_']
['_', '_', '_']
['_', '_', '_']


 Game table:
['x', '0', '_']
['_', '_', '_']
['_', '_', '_']

row: 1
col: 1

 Game table:
['x', '0', '_']
['_', 'x', '_']
['_', '_', '_']


 Game table:
['x', '0', '0']
['_', 'x', '_']
['_', '_', '_']

row: 2
col: 2

 Game table:
['x', '0', '0']
['_', 'x', '_']
['_', '_', 'x']

x


In [8]:
if __name__ == '__main__':
    main()

Current Table
['_', '_', '_']
['_', '_', '_']
['_', '_', '_']

Choose the dificulty level:
1.Easy
2.Medium
3.Hard
level:3
row: 1
col: 1

 Game table:
['_', '_', '_']
['_', 'x', '_']
['_', '_', '_']


 Game table:
['0', '_', '_']
['_', 'x', '_']
['_', '_', '_']

row: 2
col: 2

 Game table:
['0', '_', '_']
['_', 'x', '_']
['_', '_', 'x']


 Game table:
['0', '_', '0']
['_', 'x', '_']
['_', '_', 'x']

row: 0
col: 1

 Game table:
['0', 'x', '0']
['_', 'x', '_']
['_', '_', 'x']


 Game table:
['0', 'x', '0']
['_', 'x', '_']
['_', '0', 'x']

row: 1
col: 2

 Game table:
['0', 'x', '0']
['_', 'x', 'x']
['_', '0', 'x']


 Game table:
['0', 'x', '0']
['0', 'x', 'x']
['_', '0', 'x']

row: 2
col: 0

 Game table:
['0', 'x', '0']
['0', 'x', 'x']
['x', '0', 'x']

TOE
